# 第2章 PyTorch基础
## 2.2 安装配置

In [1]:
#运行文件test_gpy.py,查看GPU的配置信息
!python test_gpu.py

Support CUDA ?:  True
tensor([10.], device='cuda:0')
tensor([[ 0.0681,  0.4792, -0.8046],
        [-1.5137,  0.9180,  1.3231]], device='cuda:0')
tensor([[10.0681, 10.4792,  9.1954],
        [ 8.4863, 10.9180, 11.3231]], device='cuda:0')
Support cudnn ?:  True


## 2.4 NumPy与Tensor

### 2.4.1  torch概述
对Tensor的操作很多，从接口的角度来划分，可以分为两类：  
1）torch.function，如torch.sum、torch.add等，  
2）tensor.function，如tensor.view、tensor.add等。  
	这些操作对大部分Tensor都是等价的，如torch.add(x,y)与x.add(y)等价。在实际使用时，可以根据个人爱好选择。
	如果从修改方式的角度，可以分为以下两类。  
1）不修改自身数据，如x.add(y),x的数据不变，返回一个新的tensor。  
2）修改自身数据，如x.add_(y)（运行符带下划线后缀），运算结果存在x中，x被修改。  
以下代码说明add与add_的区别。

In [2]:
import torch

x=torch.tensor([1,2])
y=torch.tensor([3,4])
z=x.add(y)
print(z)
print(x)
x.add_(y)
print(x)

tensor([4, 6])
tensor([1, 2])
tensor([4, 6])


### 2.4.2 创建tensor
新建Tensor的方法很多，可以把列表或ndarray等数据对象直接转换为Tensor，也可以根据指定的形状构建。常见的构建Tensor的方法，可参考表2-1。

|表2-1 常见的新建Tensor方法|
|--------------------------|

|函数	|功能|
|:-------------|:---------------------------------------------------|
Tensor(*size)	|直接从参数构造一个的张量，支持list、numpy数组|
eye(row, column)	|创建指定行数，列数的二维单位tensor|
linspace(start,end,steps)	|从step到end，均匀切分成steps份|
logspace(start,end,steps)	|从10^step, 到10^end，均匀切分成steps份|
rand/randn(*size)	|生成[0,1)均匀分布/标准正态分布数据|
ones(*size)	|返回指定shape的张量，元素初始为1|
zeros(*size)	|返回指定shape的张量，元素初始为0|
ones_like(t)	|返回与t的shape相同的张量，且元素初始为1|
zeros_like(t)	|返回与t的shape相同的张量，且元素初始为0|
arange(start,end,step)	|在区间[start,end)上以间隔step生成一个序列张量|
from_numpy(ndarray) 	|从ndarray创建一个tensor|

下面举例说明。

In [3]:
import torch

#根据list数据生成tensor
torch.Tensor([1,2,3,4,5,6])
#根据指定形状生成tensor
torch.Tensor(2,3)
#根据给定的tensor的形状
t=torch.Tensor([[1,2,3],[4,5,6]])
#查看tensor的形状
t.size()
#shape与size()等价方式
t.shape
#根据已有形状创建tensor
torch.Tensor(t.size())

tensor([[0., 0., 0.],
        [0., 0., 0.]])

【说明】注意torch.Tensor与torch.tensor的几点区别  
1）torch.Tensor是torch.empty和torch.tensor之间的一种混合，但是，当传入数据时，torch.Tensor使用全局默认dtype（FloatTensor），torch.tensor从数据中推断数据类型。  
2）torch.tensor(1)返回一个固定值1，而torch.Tensor(1)返回一个大小为1的张量，它是随机初始化的值。  
举例如下。

In [4]:
#torch.tensor与torch.Tentor的区别
import torch
t1=torch.Tensor(1)
t2=torch.tensor(1)
print("t1的值{},t1的数据类型{}".format(t1,t1.type()))
print("t2的值{},t2的数据类型{}".format(t2,t2.type()))

t1的值tensor([0.]),t1的数据类型torch.FloatTensor
t2的值1,t2的数据类型torch.LongTensor


t1的值tensor([3.5731e-20]),t1的数据类型torch.FloatTensor  
t2的值1,t2的数据类型torch.LongTensor  
下面来看一些根据一定规则，自动生成tensor的例子。  

In [5]:
import torch

#生成一个单位矩阵
torch.eye(2,2)
#自动生成全是0的矩阵
torch.zeros(2,3)
#根据规则生成数据
torch.linspace(1,10,4)
#生成满足均匀分布随机数
torch.rand(2,3)
#生成满足标准分布随机数
torch.randn(2,3)
#返回所给数据形状相同，值全为0的张量
torch.zeros_like(torch.rand(2,3))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

### 2.4.3 修改tensor的形状
在处理数据、构建网络层等过程中，我们经常需要了解Tensor的形状、改变Tensor的形状。与改变NumPy的形状类似，改变tenor的形状也有很多类似函数，具体可参考表2-2。  	                     

|表2-2 为tensor常用修改形状的函数。|
|----------------------------------|

|函数	|说明|
|:--------|:---------------------------------------------------|
|size()	|返回张量的shape属性值,与函数shape(0.4版新增)等价|
|numel(input)	|计算tensor的元素个数|
|view(*shape)	| 修改tensor的shape，与reshape(0.4版新增)类似，但view返回的对象与源tensor共享内存，修改一个另一个同时修改。Reshape将生成新的tensor，而且不要求源tensor是连续的。View(-1)展平数组。|
|resize	|类似于view，但在size超出时会重新分配内存空间|
|item	|若tensor为单元素，则返回pyton的标量|
|unsqueeze	|在指定维度增加一个"1"|
|squeeze	|在指定维度压缩一个"1"|

下面来看一些实例。

In [6]:
import torch

#生成一个形状为2x3的矩阵
x = torch.randn(2, 3)
#查看矩阵的形状
x.size()   #结果为torch.Size([2, 3])

#查看x的维度
x.dim()    #结果为2
#把x变为3x2的矩阵
x.view(3,2)
#把x展平为1维向量
y=x.view(-1)  
y.shape
#添加一个维度
z=torch.unsqueeze(y,0)
#查看z的形状
z.size()   #结果为torch.Size([1, 6])
#计算Z的元素个数
z.numel()   #结果为6

6

### 2.4.4 索引操作
	Tensor的索引操作与NumPy类似，一般情况下索引结果与源数据共享内存。从tensor获取元素除了可以通过索引，也可借助一些函数，常用的选择函数可参考表2-3。
|表2-3 常用选择操作函数|
|----------------------|

|函数	|说明|
|:---------------------------|:-------------------------------------|
index_select(input,dim,index)	|在指定维度上选择一些行或列|
nonzero(input)	|获取非0元素的下标|
masked_select(input,mask)	|使用二元值进行选择|
gather(input,dim,index)	|在指定维度上选择数据,输出的形状与index（index的类型必须是LongTensor类型的）一致|
scatter_( input, dim, index, src)	|为gather的反操作，根据指定索引补充数据|

以下为部分函数的实现代码：

In [7]:
import torch


#设置一个随机种子
torch.manual_seed(100) 
#生成一个形状为2x3的矩阵
x = torch.randn(2, 3)
#根据索引获取第1行，所有数据
x[0,:]
#获取最后一列数据
x[:,-1]
#生成是否大于0的Byter张量
mask=x>0
#获取大于0的值
torch.masked_select(x,mask)
#获取非0下标,即行，列索引
torch.nonzero(mask)
#获取指定索引对应的值,输出根据以下规则得到
#out[i][j] = input[index[i][j]][j]  # if dim == 0
#out[i][j] = input[i][index[i][j]]  # if dim == 1
index=torch.LongTensor([[0,1,1]])
torch.gather(x,0,index)
index=torch.LongTensor([[0,1,1],[1,1,1]])
a=torch.gather(x,1,index)
#把a的值返回到一个2x3的0矩阵中
z=torch.zeros(2,3)
z.scatter_(1,index,a)

tensor([[ 0.3607, -0.2859,  0.0000],
        [ 0.0000, -1.3833,  0.0000]])

### 2.4.5 广播机制
前文1.8节介绍了NumPy的广播机制，它是向量运算的重要技巧。PyTorch也支持广播规则，下面通过几个示例进行说明。

In [8]:
import torch
import numpy as np

A = np.arange(0, 40,10).reshape(4, 1)
B = np.arange(0, 3)
#把ndarray转换为Tensor
A1=torch.from_numpy(A)  #形状为4x1
B1=torch.from_numpy(B)  #形状为3
#Tensor自动实现广播
C=A1+B1
#我们可以根据广播机制，手工进行配置
#根据规则1，B1需要向A1看齐，把B变为（1,3）
B2=B1.unsqueeze(0)  #B2的形状为1x3
#使用expand函数重复数组，分别的4x3的矩阵
A2=A1.expand(4,3)
B3=B2.expand(4,3)
#然后进行相加,C1与C结果一致
C1=A2+B3
print(C1)

tensor([[ 0,  1,  2],
        [10, 11, 12],
        [20, 21, 22],
        [30, 31, 32]], dtype=torch.int32)


### 2.4.6 遂元素操作
与NumPy一样，tensor也有逐元素操作，操作内容相似，但使用函数可能不尽相同。大部分数学运算都属于逐元操作，逐元素操作输入与输出的形状相同。，常见的逐元素操作，可参考表2-4。

|表2-4常见逐元素操作|
|-------------------|
                    
|函数	|说明|
|:-------------------|:-------------------------------|
abs/add	|绝对值/加法|
addcdiv(t,t1,t2,value=1)	|t1与t2的按元素除后，乘value加t|
addcmul(t,t1,t2, value=1)	|t1与t2的按元素乘后，乘value加t|
ceil/floor	|向上取整/向下取整|
clamp(t, min, max)	|将张量元素限制在指定区间|
exp/log/pow	|指数/对数/幂|
mul(或*)/neg	|逐元素乘法/取反|
sigmoid/tanh/softmax	|激活函数|
sign/sqrt	|取符号/开根号|

【说明】这些操作均创建新的tensor，如果需要就地操作，可以使用这些方法的下划线版本，例如abs_。  
以下为部分逐元素操作代码实例。


In [9]:
import torch

t = torch.randn(1, 3)
t1 = torch.randn(3, 1)
t2 = torch.randn(1, 3)
#t+0.1*(t1/t2)
torch.addcdiv(t, t1, t2,value=0.1)
#计算sigmoid
torch.sigmoid(t)
#将t限制在[0,1]之间
torch.clamp(t,0,1)
#t+2进行就地运算
t.add_(2)

tensor([[1.6828, 1.1340, 3.7482]])

### 2.4.7 归并操作
归并操作，顾名思义，就是对输入进行归并或合计等操作，这类操作的输入输出形状一般不相同，而且往往是输入大于输出形状。归并操作可以对整个tensor进行归并，也可以沿着某个维度进行归并。常见的归并操作可参考表2-5。

|表2-5 常见的归并操作|
|--------------------|

|函数	|说明|
|:--------------|:----------------------|
cumprod(t, axis)	|在指定维度对t进行累积|
cumsum	|在指定维度对t进行累加|
dist(a,b,p=2)	|返回a,b之间的p阶范数|
mean/median	|均值/中位数|
std/var	|标准差/方差|
norm(t,p=2)	|返回t的p阶范数|
prod(t)/sum(t)	|返回t所有元素的积/和|

【说明】
	归并操作一般涉及一个dim参数，指定沿哪个维进行归并。另一个参数是keepdim，说明输出结果中是否保留维度1，默认情况是False，即不保留。  
以下为归并操作的部分代码。

In [11]:
import torch

#生成一个含6个数的向量
a=torch.linspace(0,10,6)
#使用view方法，把a变为2x3矩阵
a=a.view((2,3))
#沿y轴方向累加，即dim=0
b=a.sum(dim=0)   #b的形状为[3]
#沿y轴方向累加，即dim=0,并保留含1的维度
b=a.sum(dim=0,keepdim=True) #b的形状为[1,3]


### 2.4.8比较操作
比较操作一般进行逐元素比较，有些是按指定方向比较。常用的比较函数可参考表2-6。

|表2-6 常用的比较函数|
|--------------------|

|函数	|说明|
|:--------|:-------------------------|
eq	|比较tensor是否相等，支持broadcast|
equal	|比较tensor是否有相同的shape与值|
ge/le/gt/lt	|大于/小于比较/大于等于/小于等于比较|
max/min(t,axis)	|返回最值，若指定axis，则额外返回下标|
topk(t,k,axis)	|在指定的axis维上取最高的K个值 | 

以下是部分函数的代码实现。

In [12]:
import torch

x=torch.linspace(0,10,6).view(2,3)
#求所有元素的最大值
torch.max(x)   #结果为10
#求y轴方向的最大值
torch.max(x,dim=0)  #结果为[6,8,10]
#求最大的2个元素
torch.topk(x,1,dim=0)  #结果为[6,8,10],对应索引为tensor([[1, 1, 1]

torch.return_types.topk(
values=tensor([[ 6.,  8., 10.]]),
indices=tensor([[1, 1, 1]]))

### 2.4.9 矩阵操作
机器学习和深度学习中存在大量的矩阵运算，用的比较多的有两种，一种是逐元素乘法，另外一种是点积乘法。PyTorch中常用的矩阵函数可参考表2-7。

|表2-7 常用矩阵函数|
|------------------|

|函数	|说明|
|:---------|:---------------------------------|
dot(t1, t2)	|计算张量(1D)的内积或点积|
mm(mat1, mat2)/bmm(batch1,batch2)	|计算矩阵乘法/含batch的3D矩阵乘法|
mv(t1, v1)	|计算矩阵与向量乘法|
t	|转置|
svd(t)	|计算t的SVD分解|

【说明】  
1）torch的dot与NumPy的dot有点不同，torch中dot对两个为1维张量进行点积运算，NumPy中的dot无此限制。  
2）mm是对2维矩阵进行点积运算，bmm对含batch的3维矩阵进行点积运算。  
3）转置运算会导致存储空间不连续，需要调用contiguous方法转为连续。  

In [13]:
import torch

a=torch.tensor([2, 3])
b=torch.tensor([3, 4])

torch.dot(a,b)  #运行结果为18
x=torch.randint(10,(2,3))
y=torch.randint(6,(3,4))
torch.mm(x,y)
x=torch.randint(10,(2,2,3))
y=torch.randint(6,(2,3,4))
torch.bmm(x,y)


tensor([[[28, 77, 30, 27],
         [24, 73, 25, 21]],

        [[15, 68, 57,  6],
         [25, 39, 43, 10]]])

### 2.4.10 PyTorch与NumPy比较
	PyTorch与NumPy有很多类似的地方，并且有很多相同的操作函数名称，或虽然函数名称不同但含义相同；当然也有一些虽然函数名称相同，但含义不尽相同。对此，有时很容易混淆，下面我们把一些主要的区别进行汇总，具体可参考表2-8。

|表2-8  PyTorch与NumPy函数对照表|
|-------------------------------|

|操作类别	|NumPy	|PyTorch|
|:----------|:--------|:-----------------------|
|数据类型	|np.ndarray	|torch.Tensor|
|	|np.float32	|torch.float32; torch.float|
|	|np.float64	|torch.float64; torch.double|
|	|np.int64	|torch.int64; torch.long|
|从已有数据构建	|np.array([3.2, 4.3], dtype=np.float16)	|torch.tensor([3.2, 4.3],dtype=torch.float16)|
|	|x.copy()|	x.clone()|
|	|np.concatenate	|torch.cat|
|线性代数|	np.dot	|torch.mm|
|属性|	x.ndim	|x.dim()|
|	|x.size	|x.nelement()|
|形状操作	|x.reshape	|x.reshape; x.view|
|	|x.flatten	|x.view(-1)|
|类型转换	|np.floor(x)	|torch.floor(x); x.floor()|
|比较|np.less	|x.lt|
|	|np.less_equal/np.greater	|x.le/x.gt|
|	|np.greater_equal/np.equal/np.not_equal|	x.ge/x.eq/x.ne|
|随机种子	|np.random.seed	|torch.manual_seed|
